# Imports

In [ ]:
import pandas as pd
import numpy as np
import altair as alt

from utils import download_tfrecord

# Tensorboard

Launch a TensorBoard Session in VS Code or...

Run the below (you may have to run the second command twice)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=log

or use

```
!tensorboard --logdir log
```
if the above doesn't work

# Datasets

Run the below cells to download the CIFAR-10 datasets (or the CIFAR-100 ones)

In [ ]:
download_tfrecord("test10", "train10")

# Training (and evaluation)

Use ```main.py``` to train on the imbalanced version of CIFAR 10. This is to get the results in Table 1.

Call it with the following flags/parameters:

* ```dataset```: ```cifar10-lt``` (for CIFAR 10)
* ```mode```: ```baseline```, ```posthoc``` or ```loss```
    * whether to use baseline (vanilla) ERM
    * or posthoc modification of logits
    * or adjusted loss function

The results can be found in Tensorboard: ...

## Experiments and Results

Vanilla ERM

In [ ]:
!python main.py --dataset=cifar10-lt --mode=baseline --tb_log_dir=log/ERM

Posthoc update

In [ ]:
!python main.py --dataset=cifar10-lt --mode=posthoc --tb_log_dir=log/Additive_update

Logit adjusted loss

In [ ]:
!python main.py --dataset=cifar10-lt --mode=loss --tb_log_dir=log/LA

### Read the results

* For Figure 1 and Table 1

#### Figure 1

In [ ]:
# Get the class probabilities

# Define the file path
file_path = "./data/cifar10-lt_base_probs.txt"

# Initialize a list to store the numbers
class_prob = []

# Open the file and read it line by line
with open(file_path, "r") as file:
    for line in file:
        # Remove leading/trailing whitespace and convert the line to a float
        number = float(line.strip())
        class_prob.append(number)

In [ ]:
# the number of samples in the training data. possible values for β

N = 10000
βs = [1, 0.999, 0.99, 0]

In [ ]:
# calculate effective class frequency with formula in terms of β and empirical class frequency

df = pd.DataFrame(
    {
        "class_num": list(range(1, 10 + 1)) * len(βs),
        "class_prob": class_prob * len(βs),
        "β": sum([[β] * 10 for β in βs], start=[]),
    }
)

df["effective_class_frequency"] = (1 - df["β"] ** (N * df["class_prob"])) / (
    1 - df["β"]
)

In [ ]:
# normalise effective class frequency to get effective class probability

df["effective_class_frequency"] = df["effective_class_frequency"] / df.groupby("β")[
    "effective_class_frequency"
].transform("sum")

# when β = 1, use empirical class probability

df["effective_class_frequency"] = np.where(
    df["effective_class_frequency"].isna(),
    df["class_prob"],
    df["effective_class_frequency"],
)

In [ ]:
# plot effective class probability for each class number for each β

alt.Chart(data=df).mark_bar().encode(
    x=alt.X("β:N", title=None, axis=alt.Axis(ticks=False, labels=False), sort=βs),
    y=alt.Y("effective_class_frequency", title="Smoothed class probability"),
    color=alt.Color("β:N", sort=βs),
    column=alt.Column("class_num:O", title="Classes (ranked by frequency)"),
)